In [ ]:
import torch
import os
from torchvision import datasets, transforms, models

In [ ]:
train_dir = os.path.join('dataset', 'part_one_dataset', 'train_data')
eval_dir = os.path.join('dataset', 'part_one_dataset', 'eval_data')

train_path = os.path.join(train_dir, '1_train_data.tar.pth')
eval_path = os.path.join(eval_dir, '1_eval_data.tar.pth')

t = torch.load(train_path, weights_only = False)


In [ ]:
from torchvision import models
import torch

# Load a pre-trained ResNet model
resnet = models.resnet18(pretrained=True)
resnet = torch.nn.Sequential(*list(resnet.children())[:-1])  # Remove the last layer
resnet.eval()  # Set to evaluation mode

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet = resnet.to(device)


/home/arush-upadhyaya/anaconda3/envs/771/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/arush-upadhyaya/anaconda3/envs/771/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
import numpy as np
import torch

# Example ndarray of shape (2500, 32, 32, 3)
data, targets = t['data'], t['targets'] # both numpy.ndarray
# Convert to PyTorch tensor
X_tensor = torch.tensor(data, dtype=torch.float32)  # Convert to tensor
X_tensor = X_tensor.permute(0, 3, 1, 2)  # Change shape to (2500, 3, 32, 32)


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 (ResNet input size)
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
])

# 2. Apply the transformation to each image in the batch
# Convert the tensor to float32 if it's not already (for normalization)
tensor = X_tensor.float()

# 3. Apply transformations to the batch of images
# The transform function expects each image to be a PIL Image,
# so we will need to loop through and apply the transformations to each image
transformed_images = []
for image in tensor:
    # Convert each image tensor (C, H, W) to PIL Image for transformation
    transformed_image = transform(image)  # Apply the transformations
    transformed_images.append(transformed_image)

# 4. Stack the transformed images back into a batch
preprocessed_tensor = torch.stack(transformed_images)  # Shape: (2500, 3, 224, 224)

# 5. Check the shape of the preprocessed tensor
print(preprocessed_tensor.shape)  

torch.Size([2500, 3, 224, 224])


In [ ]:
device = torch.device("cpu") # for apple. aap apna dekh lijye
resnet = resnet.to(device)

preprocessed_batch = preprocessed_tensor.to(device)

# 4. Get the embeddings (feature maps)
with torch.no_grad():  # Disable gradients for inference
    feature_maps = resnet(preprocessed_batch)  # Shape will be (batch_size, 512, 1, 1)

# 5. Flatten the feature maps (optional)
embeddings = feature_maps.view(feature_maps.size(0), -1)  # Flatten to shape (batch_size, embedding_size)

# Print the embeddings shape (should be 2500 x 512)
print("Embeddings shape:", type(embeddings))

In [ ]:
def learning_with_prototype(X_train, y_train, X_test):

    # needs to be tweaked atm
    """
    Learning with Prototype (LwP) classifier.

    Parameters:
        X_train (ndarray): Training data features.
        y_train (ndarray): Training data labels.
        X_test (ndarray): Test data features.

    Returns:
        ndarray: Predicted labels for the test data.
    """
    unique_labels = np.unique(y_train)
    class_prototypes = [np.mean(X_train[y_train == label], axis=0) for label in unique_labels]
    predicted_labels = [unique_labels[np.argmin(np.linalg.norm(sample - class_prototypes, axis=1))] for sample in X_test]
    return np.array(predicted_labels)
